In [1]:
import pandas as pd
import numpy as np
import git
from datetime import datetime as dt
from connect_to_mysql import Database
import networkx as nx
import matplotlib.pyplot as plt
import collections 
pd.set_option("display.max_columns", None)

In [ ]:
#repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
#df = pd.read_csv(f"{repo}/data/cleaning/test_1651834681.csv")
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]

In [ ]:
Database.query_dataframe('rides', 'Show tables')

In [ ]:
#df_edges = Database.query_dataframe('stopconnection', "Select * From stopconnection")
df_edges = pd.read_excel("/Users/ericchittka/Documents/Daten/MoDstops+Preismodell.xlsx")

In [ ]:
df_edges.dtypes

### Number of Drives 

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["scheduled_to"])[0].month

In [ ]:
frequencies = pd.DataFrame(df[(pd.to_datetime(df["scheduled_to"]).dt.month == 2) & (pd.to_datetime(df["scheduled_to"]).dt.year == 2022) ].groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()

frequencies

In [ ]:
frequencies["pickup_address"] = frequencies["pickup_address"].astype(str)
frequencies["dropoff_address"] = frequencies["dropoff_address"].astype(str)

In [ ]:
#df_edges = pd.read_excel(
#        f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
#)

In [ ]:
df_edges.columns

In [ ]:
df_edges = df_edges[["id_start", "id_end"]]

In [ ]:
df_edges["id_start"] = df_edges["id_start"].astype(str)
df_edges["id_end"] = df_edges["id_end"].astype(str)

In [ ]:
edges = pd.merge(df_edges, frequencies,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "number_of_drives"]]
edges = edges[edges["number_of_drives"].isna() == False]
edges["number_of_drives"] = 28  / edges["number_of_drives"] 

In [ ]:
edges

In [ ]:
edges["number_of_drives"].fillna(0, inplace=True)

In [ ]:
#G = nx.from_pandas_edgelist(
#    edges,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "number_of_drives"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G)
#plt.show()

In [ ]:
#print(nx.shortest_path(G, source="10002", target="4007", weight = "number_of_drives"))
#print(nx.shortest_path_length(G, source="10002", target="4007", weight="number_of_drives", method='dijkstra'))

### Average Ride Time per drive  

In [ ]:
#df
#df.dtypes['ride_time']

In [ ]:
average = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean())
average.rename(columns = {"ride_time":"avg_ride_time"}, inplace = True)
average

In [ ]:
average["pickup_address"] = average["pickup_address"].astype(str)
average["dropoff_address"] = average["dropoff_address"].astype(str)

In [ ]:
edges2 = pd.merge(df_edges, average,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "avg_ride_time"]]
edges2 = edges2[edges2["avg_ride_time"].isna() == False]
edges2

In [ ]:
#G1 = nx.from_pandas_edgelist(
#    edges2,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "avg_ride_time"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G1)
#plt.show()

In [ ]:
#print(nx.shortest_path(G1, source="1001", target="9015", weight = "avg_ride_time"))
#print(nx.shortest_path_length(G1, source="1001", target="9015", weight="avg_ride_time", method='dijkstra'))

### Original Function

In [ ]:
def get_shortest_ride(startpoint, endpoint, start_date, end_date):
    start_date = dt.strptime(start_date, "%m/%d/%Y")
    end_date = dt.strptime(end_date, "%m/%d/%Y")
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )

    path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
    shortest_time = nx.shortest_path_length(G, source=startpoint, target=endpoint, weight="avg_time_to_destination", method='dijkstra') 

    return (path, shortest_time)

### Set start and end date 

In [2]:
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/cleaning/test_1656252990.csv")
df = df[(df["state"] == "completed")]

In [3]:
#df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
startdate = df.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = df.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [4]:
#df['pickup_address'] = df['pickup_address'].map(str)
#df['dropoff_address'] = df['dropoff_address'].map(str)

### Drive DF

In [5]:
days = (end_date - start_date).days + 1
drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
drives["waiting_time"] = days / drives["number_of_drives"]
drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

In [6]:
drives

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1004,5,54.800000,0.001738,54.801738
1,1001,1005,18,15.222222,0.001552,15.223774
2,1001,1008,3,91.333333,0.003538,91.336871
3,1001,1012,1,274.000000,0.007627,274.007627
4,1001,1013,1,274.000000,0.005856,274.005856
...,...,...,...,...,...,...
2271,18005,7001,4,68.500000,0.006704,68.506704
2272,18005,9001,1,274.000000,0.010185,274.010185
2273,18005,9005,6,45.666667,0.009236,45.675903
2274,18005,9006,1,274.000000,0.009155,274.009155


### Graph

In [7]:
G = nx.from_pandas_edgelist(
    drives,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [8]:
G.nodes

NodeView((1001, 1004, 1005, 1008, 1012, 1013, 4018, 6004, 9005, 9015, 9019, 12011, 13003, 13005, 14006, 1002, 2007, 3018, 3023, 3024, 4007, 4008, 4025, 4027, 4041, 6002, 9006, 10004, 11012, 12010, 15013, 16009, 16027, 1003, 9018, 1007, 1014, 2008, 3001, 3004, 3010, 3021, 3025, 4006, 4037, 5001, 5002, 5003, 8001, 9001, 9002, 9009, 9011, 9021, 11002, 11003, 11005, 11008, 11017, 12003, 12004, 12005, 12007, 12008, 13001, 13002, 14003, 15007, 15009, 15011, 15012, 15015, 17007, 18005, 1006, 2002, 2005, 2006, 3006, 3009, 3011, 3013, 3014, 3015, 3019, 3020, 3026, 3027, 3028, 4001, 4013, 4024, 4026, 4029, 4031, 4033, 4034, 4035, 4039, 5006, 5009, 6003, 7001, 9004, 9017, 10001, 10003, 10005, 11007, 11009, 11011, 11014, 12001, 12009, 14002, 14011, 15002, 15004, 15005, 15006, 15014, 1009, 1010, 3002, 3016, 4030, 9003, 9010, 1011, 3007, 3017, 4012, 5007, 5008, 5010, 8002, 12012, 13004, 14005, 2001, 2004, 2009, 3012, 4004, 4016, 4020, 4023, 4038, 4040, 6001, 6005, 9008, 11010, 12002, 14001, 15001, 1

In [9]:
def calculate_graph(start_date, end_date):
  
    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )
    return G

In [10]:
def get_shortest_ride(startpoint, endpoint, G):

    if startpoint not in G:
        return 'Not in G'
    elif endpoint not in G: 
        return 'Not in G'
    else:
        path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
        return path

In [11]:
# df_test = pd.DataFrame({'Start':['1001', '1002', '1931391'], 'End': ['10004', '1008', '32432432']})
# df_test["Start"] = df_test["Start"].astype(str).astype(int)
# df_test["End"] = df_test["End"].astype(str).astype(int)
# df_test['HotSpots'] = df_test.apply(lambda x: get_shortest_ride(x.Start, x.End, G), axis=1)

In [12]:
df_edges = pd.read_excel("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

In [13]:
df_edges.rename(columns= {'Start #':'start_id', 'Ende #':'end_id'}, inplace = True)
df_edges['Spots'] = df_edges.apply(lambda x: get_shortest_ride(x.start_id, x.end_id, G), axis=1)

In [14]:
df_edges

,start_id,Start Name,end_id,Ende Name,Route [m],Luftlinie [m],VRN-eTarif\nohne Bahncard,VRN-eTarif\nmit BahnCard,Qualitätszuschlag,MoD-Fahrpreis \nohne BahnCard,MoD-Fahrpreis\n mit Bahncard,0.12680550240634963,0.07159177456207169,0.18042226487523982,0.5159630940478845,0.2894135567402895,0.72936660268714,price_mod,price_mod_2pers,price_mod_3pers,price_mod_4pers,price_bahncard,price_bahncard_2pers,price_bahncard_3pers,price_bahncard_4pers,price_vrn_surcharge,price_vrn_surcharge_2pers,price_vrn_surcharge_3pers,price_vrn_surcharge_4pers,Spots
0,1001,Mandelgasse,1002,Hauptfeuerwache,366,285,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460,"[1001, 1008, 1002]"
1,1001,Mandelgasse,1003,Kindergarten St. Marien,994,586,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460,"[1001, 1005, 9018, 1003]"
2,1001,Mandelgasse,1004,Strohmarkt,796,694,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460,"[1001, 1005, 4025, 1004]"
3,1001,Mandelgasse,1005,Rathaus,555,510,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460,"[1001, 1005]"
4,1001,Mandelgasse,1006,Villenstraße,1416,947,1.65,1.24,1.175,2.825,2.415,0.145133,NaN,NaN,0.584071,NaN,NaN,2.825,4.825,6.825,8.825,2.415,4.415,6.005,7.800,1.175,3.175,4.765,6.560,"[1001, 1005, 4025, 1006]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40195,18005,Festhalle Geinsheim,15014,Loblocher Straße,14100,10281,4.15,3.14,5.605,9.755,8.745,0.103537,NaN,NaN,0.425423,NaN,NaN,9.755,17.435,25.115,32.795,8.745,16.425,23.095,30.270,5.605,13.285,19.955,27.130,"[18005, 7001, 1009, 4025, 1007, 15014]"
40196,18005,Festhalle Geinsheim,15015,Am Krautgarten,12650,8787,3.65,2.76,5.335,8.985,8.095,0.099054,NaN,NaN,0.406233,NaN,NaN,8.985,16.145,23.305,30.465,8.095,15.255,21.525,28.240,5.335,12.495,18.765,25.480,"[18005, 7001, 1009, 2007, 6003, 15015]"
40197,18005,Festhalle Geinsheim,16009,In den Obergärten,6933,5236,2.90,2.19,2.950,5.850,5.140,0.121368,NaN,NaN,0.495726,NaN,NaN,5.850,10.250,14.650,19.050,5.140,9.540,13.230,17.275,2.950,7.350,11.040,15.085,"[18005, 7001, 1009, 4025, 16009]"
40198,18005,Festhalle Geinsheim,16027,Altes Schulhaus Lachen,5694,4883,2.65,2.00,2.485,5.135,4.485,0.126582,NaN,NaN,0.516066,NaN,NaN,5.135,8.945,12.755,16.565,4.485,8.295,11.455,14.940,2.485,6.295,9.455,12.940,"[18005, 7001, 1008, 1002, 16027]"


In [15]:
hotSpots_list = list(df_edges['Spots'])

In [17]:
flat_hotSpots_list = [item for sublist in hotSpots_list for item in sublist]

In [19]:
counter=collections.Counter(flat_hotSpots_list)
print(counter.most_common(20))

[(1008, 20448), (4025, 20178), (1005, 10060), (1009, 6490), (' ', 4740), (1002, 4170), (1007, 4072), (12007, 3008), (7001, 2872), (6004, 2478), (14003, 2418), (3001, 2410), ('N', 2370), ('o', 2370), ('t', 2370), ('i', 2370), ('n', 2370), ('G', 2370), (11017, 2216), (1010, 1810)]
